# ***$$技术报告$$***
<center>碱基配对</center>

#### 一、注意事项

比赛指定python版本==3.8\
必须使用torch框架,gpu/cpu版本皆可\
cuda版本不限（若有）\
不限使用其他的库

选手只可上交一个"技术报告.ipynb"文件，该文件包含选手从序列到活性预测的全过程代码+说明（如下框架）\
若有其余数据，则需要放在同一个目录，一同打包为zip文件上传,代码中数据路径为相对路径

算力资源需要选手自行解决\
注意该程序的可复现性，审核人员会对程序进行复现操作\
本程序上交主办方后，主办方在三个月（自上交之日算起）内会对其内容保密。三个月后会公开优秀算法，以供后续大家学习



#### 二、包的安装
选手需要把配置（environment）写明，下面是例子\
pytorch==2.4.1\
pytorch-cuda==12.1\
pandas==2.0.3\
numpy==1.24.3\
scikit-learn==1.3.2\
biopython==1.78


#### 三、依赖库

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

**确定随机数种子**  
保证每次运行结果相同


In [9]:
#确定随机种子，保证结果的可重复性
seed=66
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

#### 四、代码主体部分（可有多个主体，按逻辑划分）

##### 完整流程概述

根据GFP蛋白质序列预测其荧光亮度，采用深度学习方法构建回归模型。整体流程包括数据读取、模型构建、训练与保存，具体步骤如下：

1. **数据读取与解析**
   - 使用 `Bio.SeqIO` 模块从FASTA格式的文件中读取氨基酸序列及其log亮度值；
   - 编写 `read_gfp_data_with_bio` 函数进行解析，提取序列列表与对应亮度。

2. **数据预处理**
   - 构建`apply_mutations_to_wt`处理要预测的数据，将其装换为序列；
   - 对氨基酸序列进行编码，将其转换为数字ID；
   - 构建 `encode_amino_acids` 函数，将氨基酸序列映射
   为数字ID列表。
   - 对数据集进行划分,训练集占比为0.8，测试集占比为0.2，训练集再划分出验证集，验证集占比为0.125，并对亮度值进行归一化处理。

   
3. **模型构建**
   - 定义神经网络类 `GFPBrightnessPredictor`，继承自 `nn.Module`；
   - 包含以下模块：
     - 嵌入层 `nn.Embedding`：将氨基酸ID映射为稠密向量；
     - 双向LSTM `nn.LSTM`：提取局部与全局序列特征；
     - 注意力机制 `nn.Sequential`：聚焦关键位置；
     - 全连接层 `nn.Linear`：输出亮度值；
     - 使用 `LayerNorm` 进行归一化增强稳定性。

4. **模型训练**
   - 构建训练数据加载器 `TensorDataset` 与 `DataLoader`；
   - 选用损失函数 `HuberLoss`；
   - 使用 `Adam` 优化器和 `ReduceLROnPlateau` 学习率调度器；
   - 迭代训练模型若干轮，每轮更新权重、评估性能。
5. **模型保存**
   - 训练完成后，将模型参数保存到 `gfp_model1.pt` 文件中。
6. **模型预测**
   - 加载保存的模型参数；
   - 对新数据进行预测，输出亮度值。
   - 排序输出

In [10]:
#读取和解析亮度序列
def read_gfp_data_with_bio(fasta_path):
    sequences = []
    brightness = []
    for record in SeqIO.parse(fasta_path, "fasta"):
        header = record.id  # 如：avGFP-3.719212132-41.23189601673761-WT
        parts = header.split('-')
        log_bright = float(parts[1])  # 第二段是log亮度
        brightness.append(log_bright)
        sequences.append(str(record.seq))  # 氨基酸序列
    return sequences, brightness
train_sequences,train_brightness = read_gfp_data_with_bio('GFP_sequences.fasta')
print("读取到{}条序列".format(len(train_sequences)))

读取到141144条序列


In [11]:
#处理要预测的数据
def apply_mutations_to_wt(wt_seq: str, mutation_str: str) -> str:
    """将突变字符串应用到 WT 序列"""
    seq = list(wt_seq)
    mutations = mutation_str.strip().split(':')
    
    for mut in mutations:
        # 提取数字部分和氨基酸部分
        pos_str = ''
        aa = ''
        for char in mut:
            if char.isdigit():
                pos_str += char
            else:
                aa = char
                break  # 一旦遇到非数字字符就停止
        
        if pos_str and aa:
            pos = int(pos_str) - 1  # 1-based 转 0-based
            if 0 <= pos < len(seq):
                seq[pos] = aa
    
    return ''.join(seq)


In [12]:
#要预测的突变
mutants = [
    '5N:73H:163G:34A:90H:206V',
    '11Y:157G:158Q',
    '5N:73H:163G:34V:117G:206M',
    '',
    '5N:73H:163G:34A:90E:206M',
    '154P:157S:214M',
    '93L:157T:158E',
    '154I:157A:214M',
    '157T:158Q:214M',
    '5N:34V:72T:73H:97I:117N:142Q:158Q:163G:171M:175G:206I:214R',
    '5N:34V:73H:117N:128T:142Q:153N:163G:171M:206I',
    '5N:73H:163G:34V:142Q:206I:90D:117N:203Y',
    '5N:73H:163G:34V:142G:206V',
    '5N:73H:163G:34V:142Q:206I:90D:117K:128V',
    '5N:34V:43S:72T:73H:117N:142Q:163G:171M:175G:206I:214R:237A',
    '5N:34V:43S:72T:73H:97G:117N:142Q:163G:171M:206I:214R',
    '5N:73H:163G:34V:142Q:206I:90E:117N:214H:63S:170N:171M:39S:43S:64M:175S',
    '5N:73H:163G:34V:142Q:206I:90D:117N:203L',
    '5N:34V:43S:72T:73H:117N:142Q:163G:171M:175A:206I:214R',
    '93L:157T:158N',
]
#野生型
wt_seq='MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'
#生成突变体序列
pred_sequences=[apply_mutations_to_wt(wt_seq,mutant) for mutant in mutants]


In [13]:
#转换模块
AA_TO_ID = {aa: i for i, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
# 定义填充标识的ID，用于在编码氨基酸序列时填充到指定长度，此处填充ID为20
PAD_ID = 20
# 定义函数，将氨基酸序列编码为整数序列
def encode_seqs(seqs, max_len=256):
    result = np.full((len(seqs), max_len), PAD_ID, dtype=np.int32)  # 提前填充好
    for i, seq in enumerate(seqs):
        for j, aa in enumerate(seq[:max_len]):
            result[i, j] = AA_TO_ID.get(aa, PAD_ID)
    return result

In [19]:
#进行装换
max_len=256
train_seqs=encode_seqs(train_sequences,max_len)
pred_seqs=encode_seqs(pred_sequences,max_len)
#将标签转换为numpy数组
train_brightness=np.array(train_brightness)
#划分训练集和测试集
train_seqs,test_seqs,train_labels,test_labels=train_test_split(train_seqs,train_brightness,test_size=0.2,random_state=0)
#标签Z-score标准化
scaler=StandardScaler()
train_labels=scaler.fit_transform(train_labels.reshape(-1,1))
test_labels=scaler.transform(test_labels.reshape(-1,1))
#划分验证集和训练集
train_seqs,val_seqs,train_labels,val_labels=train_test_split(train_seqs,train_labels,test_size=0.125,random_state=0)
#将数据转换为PyTorch张量

train_seq = torch.tensor(train_seqs, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.float32).squeeze(-1)

val_seq = torch.tensor(val_seqs, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.float32).squeeze(-1)

test_seq = torch.tensor(test_seqs, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.float32).squeeze(-1)

train_dataset = TensorDataset(train_seq, train_labels)
val_dataset = TensorDataset(val_seq, val_labels)
test_dataset = TensorDataset(test_seq, test_labels)

batch_size=128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


In [20]:
#定义神经网络
class GFPBrightnessPredictor(nn.Module):
    def __init__(self, vocab_size=21, embed_dim=64, hidden_dim=128):
        super().__init__()
        # 1. 嵌入层：将序列中的数字转为稠密向量
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=PAD_ID)
        # 归一化
        self.emb_norm = nn.LayerNorm(embed_dim)

        # 2. 双向 LSTM：捕捉局部 + 全局序列特征
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)

        # 3. 注意力池化聚焦关键位点
        self.attention=nn.Sequential(
            nn.Linear(hidden_dim*2,64),
            nn.Tanh(),
            nn.Linear(64,1),
        )

        # 4. 全连接层：回归亮度
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 128),  # 因为是双向 LSTM，维度是 2*hidden
            nn.ReLU(),
            nn.Dropout(0.3),#每次丢弃概率为0.3，防止过拟合
            nn.Linear(128, 1)
        )
        self.dropout=nn.Dropout(0.3)
    def forward(self, x):
        # 输入 x 的维度为 (batch_size, seq_len)，表示一批输入序列，每个序列长度为 seq_len
        x = self.embedding(x)  # 经过嵌入层处理后，输出维度变为 (batch_size, seq_len, embed_dim)
        x = self.emb_norm(x)  # 对嵌入层的输出进行归一化处理
        out, _ = self.lstm(x)  # 经过双向 LSTM 层处理后，输出维度变为 (batch_size, seq_len, hidden_dim*2)，因为是双向 LSTM，所以维度翻倍
        out = self.dropout(out) #每次丢弃概率为0.3，防止过拟合
        # 进行注意力池化操作，聚焦序列中的关键位点
        attention=self.attention(out).softmax(dim=1)  # 经过注意力网络处理并在维度 1 上进行 softmax 操作，输出维度为 (batch_size, seq_len, 1)
        pooled=(out*attention).sum(dim=1)  # 通过注意力权重对 LSTM 输出进行加权求和，得到池化后的结果
        return self.fc(pooled).squeeze(-1)  # 经过全连接层处理并去掉最后一个维度，输出维度为 (batch,)，得到最终的预测结果


#### 模型原理  
1.序列编码：将 21 种氨基酸映射为 64 维嵌入向量，捕捉残基间语义关系（如极性、疏水性）。  
2.双向 LSTM 建模：通过门控机制捕获序列前后文依赖，提取局部和全局特征（如关键功能域）。  
3.注意力聚焦：自适应权重分配机制，自动识别对荧光起决定作用的残基（如 Ser65、Tyr66）。  
4.非线性回归：多层感知器将序列特征映射到连续亮度值，捕捉残基间复杂物理化学相互作用。

In [21]:
# 早停策略
class EarlyStopping:
    def __init__(self, patience=5, delta=1e-4, path='gfp_model1.pt'):
        self.patience = patience
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.path)  # 保存当前最优模型
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GFPBrightnessPredictor().to(device)
#使用Huber损失函数，结合了均方误差（MSE）和绝对误差（MAE）的优点。
loss_fn = nn.HuberLoss(delta=1.0)

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
# 学习率调度器
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
#早停策略
early_stopping = EarlyStopping(patience=5, delta=1e-3, path='gfp_model1.pt')
epochs = 50
for epoch in range(epochs):
    # 训练
    model.train()
    train_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
    train_loss /= len(train_loader.dataset)

    # 验证集上的损失
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            val_loss += loss.item() * x.size(0)
    val_loss /= len(val_loader.dataset)
    # 学习率调整
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1} | 当前学习率: {current_lr:.6f} ",end='|')
    print(f" Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    # 早停策略
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("验证集性能停止提升，触发早停！")
        break


Epoch 1 | 当前学习率: 0.001000 | Train Loss: 0.3053 | Val Loss: 0.2617
Epoch 2 | 当前学习率: 0.001000 | Train Loss: 0.2408 | Val Loss: 0.1800
Epoch 3 | 当前学习率: 0.001000 | Train Loss: 0.1711 | Val Loss: 0.1291
Epoch 4 | 当前学习率: 0.001000 | Train Loss: 0.1210 | Val Loss: 0.0904
Epoch 5 | 当前学习率: 0.001000 | Train Loss: 0.0925 | Val Loss: 0.0707
Epoch 6 | 当前学习率: 0.001000 | Train Loss: 0.0765 | Val Loss: 0.0638
Epoch 7 | 当前学习率: 0.001000 | Train Loss: 0.0691 | Val Loss: 0.0587
Epoch 8 | 当前学习率: 0.001000 | Train Loss: 0.0614 | Val Loss: 0.0635
Epoch 9 | 当前学习率: 0.001000 | Train Loss: 0.0557 | Val Loss: 0.0530
Epoch 10 | 当前学习率: 0.001000 | Train Loss: 0.0545 | Val Loss: 0.0441
Epoch 11 | 当前学习率: 0.001000 | Train Loss: 0.0480 | Val Loss: 0.0449
Epoch 12 | 当前学习率: 0.001000 | Train Loss: 0.0479 | Val Loss: 0.0417
Epoch 13 | 当前学习率: 0.001000 | Train Loss: 0.0520 | Val Loss: 0.0453
Epoch 14 | 当前学习率: 0.001000 | Train Loss: 0.0449 | Val Loss: 0.0417
Epoch 15 | 当前学习率: 0.001000 | Train Loss: 0.0437 | Val Loss: 0.0441
Epoc

In [27]:
#测试
model.load_state_dict(torch.load('gfp_model1.pt'))

# 测试集评估
model.eval()
test_loss = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = loss_fn(pred, y)
        test_loss += loss.item() * x.size(0)
test_loss /= len(test_loader.dataset)

print(f"测试集损失: {test_loss:.4f}")


C:\Users\32840\AppData\Local\Temp\ipykernel_40756\3669852407.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('gfp_model1.pt'))


测试集损失: 0.0295


In [32]:
#加载进行预测
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_len=256
model = GFPBrightnessPredictor().to(device)
model.load_state_dict(torch.load("gfp_model1.pt"))
model.eval()
# 进行预测
encode_preds=encode_seqs(pred_sequences,max_len)
preds=torch.tensor(encode_preds,dtype=torch.long).to(device)
with torch.no_grad():
    predictions = model(preds)
    predictions=scaler.inverse_transform(predictions.cpu().numpy().reshape(-1, 1))
    predictions = predictions.squeeze().tolist()
    print("每个序列的预测亮度(log)")
    for i, pred in enumerate(predictions):
        print(f"序列 {i+1}: {pred}")
    sorted_indices = np.argsort(predictions)
    sorted_indices_list = (sorted_indices+1).tolist()
    print("按照预测值大小排序的序号列表:")
    print(sorted_indices_list)
    

每个序列的预测亮度(log)
序列 1: 3.6704061031341553
序列 2: 3.6637730598449707
序列 3: 3.693673610687256
序列 4: 3.722771644592285
序列 5: 3.692875385284424
序列 6: 3.661938428878784
序列 7: 3.559307336807251
序列 8: 3.6814284324645996
序列 9: 3.704284429550171
序列 10: 3.6232423782348633
序列 11: 3.6284070014953613
序列 12: 3.6079185009002686
序列 13: 3.6657724380493164
序列 14: 3.653627872467041
序列 15: 3.6309969425201416
序列 16: 3.6186511516571045
序列 17: 3.494438648223877
序列 18: 3.6082353591918945
序列 19: 3.6251914501190186
序列 20: 3.6141223907470703
按照预测值大小排序的序号列表:
[17, 7, 12, 18, 20, 16, 10, 19, 11, 15, 14, 6, 2, 13, 1, 8, 5, 3, 9, 4]


C:\Users\32840\AppData\Local\Temp\ipykernel_40756\832542042.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("gfp_model1.pt"))


#### 五、模型运行结果（即排序结果，需要和提交的表格中数据对应！）
4\
9\
3\
5\
8\
1\
13\
2\
6\
14\
15\
11\
19\
10\
16\
20\
18\
12\
7\
17


#### 六、参考资料

[1]https://jupyter.org/ <br>
[2]pytorch教程 https://www.runoob.com/pytorch/pytorch-tutorial.html<br>
[3]Hu, H.; Li, Z.; Elofsson, A.; Xie, S. A Bi-LSTM Based Ensemble Algorithm for Prediction of Protein Secondary Structure. Appl. Sci. 2019, 9, 3538. https://doi.org/10.3390/app9173538<br>
[4]Wang.; Generating New Protein Sequences by Using Dense Network and Attention Mechanism
Wang et al.<br>
[5]深度学习模型：LSTM (Long Short-Term Memory) - 长短时记忆网络详解 https://blog.csdn.net/2301_80840905/article/details/144107247<br>
[6]神经网络算法 —— Embedding（嵌入）！！ https://blog.csdn.net/leonardotu/article/details/136165819